In [ ]:
package_path = '../input/pytorch-image-models/pytorch-image-models-master'
import sys; sys.path.append(package_path)

from glob import glob
from sklearn.model_selection import GroupKFold, StratifiedKFold
import cv2
from skimage import io
import torch
from torch import nn
import os
from datetime import datetime
import time
import random
import cv2
import torchvision
from torchvision import transforms
import pandas as pd
import numpy as np
from tqdm import tqdm

import matplotlib.pyplot as plt
from torch.utils.data import Dataset,DataLoader
from torch.utils.data.sampler import SequentialSampler, RandomSampler
from  torch.cuda.amp import autocast, GradScaler

import sklearn
import warnings
import joblib
from sklearn.metrics import roc_auc_score, log_loss
from sklearn import metrics
import warnings
import cv2
import pydicom
import timm
from scipy.ndimage.interpolation import zoom
from sklearn.metrics import log_loss

train = pd.read_csv('../input/cassava-leaf-disease-classification/train.csv')
submission = pd.read_csv('../input/cassava-leaf-disease-classification/sample_submission.csv')

In [ ]:
CFG = {
    'normalize_mean':[0.42984136, 0.49624753, 0.3129598],
    'normalize_std':[0.21417203, 0.21910103, 0.19542212],
    'device': 'cuda:0',
    'fold_num': 5,
    'seed': 42,
    'valid_bs': 32,
    'num_workers': 4,

    'model_arch': ['tf_efficientnet_b4_ns',
                   'tf_efficientnet_b4_ns',
                   'tf_efficientnet_b4_ns',
                   'tf_efficientnet_b4_ns',
                   'tf_efficientnet_b4_ns',
                  ], #### ["tf_efficientnet_b4_ns", "vit_large_patch16_384", "deit_base_patch16_384", 'resnext50_32x4d', 'vit_base_patch16_384']
    'img_size': 512,
    'tta': 3,  ####
    'fold_list':[0],
    'used_epochs': ["../input/train3012/tf_efficientnet_b4_ns_3012_fold_0_14",
                    "../input/train3012/tf_efficientnet_b4_ns_3012_fold_1_14",
                    "../input/train3012/tf_efficientnet_b4_ns_3012_fold_2_14",
                    "../input/train3012/tf_efficientnet_b4_ns_3012_fold_3_9",
                    "../input/train3012/tf_efficientnet_b4_ns_3012_fold_4_12"], ####
    'weights': [1,1,1,1,1], ####
} 

# Helper Functions

In [ ]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    torch.cuda.manual_seed_all(seed)
    
def get_img(path):
    im_bgr = cv2.imread(path)
    im_rgb = im_bgr[:, :, ::-1]
    return im_rgb

# Dataset

In [ ]:
class CassavaDataset(Dataset):
    def __init__(
        self, df, data_root, transforms=None, output_label=True
    ):
        
        super().__init__()
        self.df = df.reset_index(drop=True).copy()
        self.transforms = transforms
        self.data_root = data_root
        self.output_label = output_label
    
    def __len__(self):
        return self.df.shape[0]
    
    def __getitem__(self, index: int):
        # get labels
        if self.output_label:
            target = self.df.iloc[index]['label']
          
        path = "{}/{}".format(self.data_root, self.df.iloc[index]['image_id'])
        
        img  = get_img(path)
        
        if self.transforms:
            img = self.transforms(image=img)['image']
            
        if self.output_label == True:
            return img, target
        else:
            return img

In [ ]:
from albumentations import (
    HorizontalFlip, VerticalFlip, IAAPerspective, ShiftScaleRotate, CLAHE, RandomRotate90,
    Transpose, ShiftScaleRotate, Blur, OpticalDistortion, GridDistortion, HueSaturationValue,
    IAAAdditiveGaussianNoise, GaussNoise, MotionBlur, MedianBlur, IAAPiecewiseAffine, RandomResizedCrop,
    IAASharpen, IAAEmboss, RandomBrightnessContrast, Flip, OneOf, Compose, Normalize, Cutout, CoarseDropout, ShiftScaleRotate, CenterCrop, Resize,
    RandomCrop
)

from albumentations.pytorch import ToTensorV2

def get_inference_transforms():
    return Compose([
        RandomCrop(512, 512),
        Transpose(p=0.5),
        HorizontalFlip(p=0.5),
        VerticalFlip(p=0.5),
        HueSaturationValue(hue_shift_limit=10, sat_shift_limit=10, val_shift_limit=10, p=0.5),
        RandomBrightnessContrast(brightness_limit=(-0.1,0.1), contrast_limit=(-0.1, 0.1), p=0.5),
        Normalize(mean=CFG["normalize_mean"], std=CFG["normalize_std"], max_pixel_value=255.0, p=1.0),
        ToTensorV2(p=1.0),
    ], p=1.)

def get_inference_transforms_vit384():
    return Compose([
        RandomCrop(384, 384),
        Transpose(p=0.5),
        HorizontalFlip(p=0.5),
        VerticalFlip(p=0.5),
        HueSaturationValue(hue_shift_limit=10, sat_shift_limit=10, val_shift_limit=10, p=0.5),
        RandomBrightnessContrast(brightness_limit=(-0.1,0.1), contrast_limit=(-0.1, 0.1), p=0.5),
        Normalize(mean=CFG["normalize_mean"], std=CFG["normalize_std"], max_pixel_value=255.0, p=1.0),
        ToTensorV2(p=1.0),
    ], p=1.)



# Model

In [ ]:
class CassvaImgClassifier(nn.Module):
    def __init__(self, model_arch, n_class, pretrained=False):
        super().__init__()
        self.model = timm.create_model(model_arch, pretrained=pretrained, num_classes=5)
    def forward(self, x):
        x = self.model(x)
        return x

# Main Loop

In [ ]:
def inference_one_epoch(model, data_loader, device):
    model.eval()
    image_preds_all = []
    pbar = tqdm(enumerate(data_loader), total=len(data_loader))
    for step, (imgs) in pbar:
        imgs = imgs.to(device).float()
        image_preds = model(imgs)   #output = model(input)
        image_preds_all += [torch.softmax(image_preds, 1).detach().cpu().numpy()]
    image_preds_all = np.concatenate(image_preds_all, axis=0)
    return image_preds_all

In [ ]:
seed_everything(CFG['seed'])
tst_preds = []
device = torch.device(CFG['device'])

test = pd.DataFrame()
test['image_id'] = list(os.listdir('../input/cassava-leaf-disease-classification/test_images/'))


for i, sub_model in enumerate(CFG['used_epochs']): 
    if "vit" not in sub_model:
        test_ds = CassavaDataset(test, '../input/cassava-leaf-disease-classification/test_images/', transforms=get_inference_transforms(), output_label=False)
    else:
        test_ds = CassavaDataset(test, '../input/cassava-leaf-disease-classification/test_images/', transforms=get_inference_transforms_vit384(), output_label=False)
        
    tst_loader = torch.utils.data.DataLoader(
        test_ds, 
        batch_size=CFG['valid_bs'],
        num_workers=CFG['num_workers'],
        shuffle=False,
        pin_memory=True,
    )

    model = CassvaImgClassifier(CFG['model_arch'][i], train.label.nunique()).to(device)
    model.load_state_dict(torch.load(sub_model, map_location=CFG['device']))
    print(sub_model)

    with torch.no_grad():
        for tta_ in range(CFG['tta']):
            tst_preds += [CFG['weights'][i]/sum(CFG['weights'])/CFG['tta']*inference_one_epoch(model, tst_loader, device)]

tst_preds = np.sum(tst_preds, axis=0)


del model
torch.cuda.empty_cache()

In [ ]:
test['label'] = np.argmax(tst_preds, axis=1)
test.to_csv('submission.csv', index=False)